In [1]:
## Import libraries and data
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/mwitiderrick/kerasDO/master/HR_comma_sep.csv")

In [2]:
## You can get a glimpse at the dataset you’re working with by using head()
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


You’ll now proceed to convert the categorical columns to numbers. You do this by converting them to dummy variables. Dummy variables are usually ones and zeros that indicate the presence or absence of a categorical feature. In this kind of situation, you also avoid the dummy variable trap by dropping the first dummy.

In [3]:
feats = ['department','salary']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

feats = ['department','salary'] defines the two columns for which you want to create dummy variables. pd.get_dummies(df,columns=feats,drop_first=True) will generate the numerical variables that your employee retention model requires. It does this by converting the feats that you define from categorical to numerical variables.

We’ve loaded in the dataset and converted the salary and department columns into a format the keras deep learning model can accept.

In [4]:
## Separating Your Training and Testing Datasets
from sklearn.model_selection import train_test_split
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

Our deep learning model expects to get the data as arrays. Therefore we use numpy to convert the data to numpy arrays with the .values attribute.

You’ll use 70% of the data for training and 30% for testing. The training ratio is more than the testing ratio because you’ll need to use most of the data for the training process. If desired, you can also experiment with a ratio of 80% for the training set and 20% for the testing set.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Transforming the Data

When building deep learning models it is usually good practice to scale our dataset in order to make the computations more efficient. In this step, we’ll scale the data using the StandardScaler; this will ensure that our dataset values have a mean of zero and a unit variable. This transforms the dataset to be normally distributed. We’ll use the scikit-learn StandardScaler to scale the features to be within the same range. This will transform the values to have a mean of 0 and a standard deviation of 1. This step is important because we’re comparing features that have different measurements; so it is typically required in machine learning.

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

We have scaled all your dataset features to be within the same range

Building the Artificial Neural Network

Now we will use keras to build the deep learning model. To do this, we’ll import keras, which will use tensorflow as the backend by default. From keras, we’ll then import the Sequential module to initialize the artificial neural network. An artificial neural network is a computational model that is built using inspiration from the workings of the human brain. We’ll import the Dense module as well, which will add layers to our deep learning model.

When building a deep learning model we usually specify three layer types:

- The input layer is the layer to which we’ll pass the features of our dataset. There is no computation that occurs in this layer. It serves to pass features to the hidden layers.
- The hidden layers are usually the layers between the input layer and the output layer—and there can be more than one. These layers perform the computations and pass the information to the output layer.
- The output layer represents the layer of our neural network that will give us the results after training our model. It is responsible for producing the output variables.

In [7]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

We’ll use Sequential to initialize a linear stack of layers. Since this is a classification problem, we’ll create a classifier variable. A classification problem is a task where we have labeled data and would like to make some predictions based on the labeled data.

In [8]:
classifier = Sequential()

In [9]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=18))

We add layers using the .add() function on our classifier and specify some parameters:

- The first parameter is the number of nodes that our network should have. The connection between different nodes is what forms the neural network. One of the strategies to determine the number of nodes is to take the average of the nodes in the input layer and the output layer.

- The second parameter is the kernel_initializer. When we fit our deep learning model the weights will be initialized to numbers close to zero, but not zero. To achieve this we use the uniform distribution initializer. kernel_initializer is the function that initializes the weights.

- The third parameter is the activation function. Our deep learning model will learn through this function. There are usually linear and non-linear activation functions. We use the relu activation function because it generalizes well on our data. Linear functions are not good for problems like these because they form a straight line.

- The last parameter is input_dim, which represents the number of features in your dataset.

In [10]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

The output layer takes the following parameters:

- The number of output nodes. We expect to get one output: if an employee leaves the company. Therefore we specify one output node.

- For kernel_initializer we use the sigmoid activation function so that we can get the probability that an employee will leave. In the event that we were dealing with more than two categories, we would use the softmax activation function, which is a variant of the sigmoid activation function.

Next, we’ll apply a gradient descent to the neural network. This is an optimization strategy that works to reduce errors during the training process. Gradient descent is how randomly assigned weights in a neural network are adjusted by reducing the cost function, which is a measure of how well a neural network performs based on the output expected from it.

The aim of a gradient descent is to get the point where the error is at its least. This is done by finding where the cost function is at its minimum, which is referred to as a local minimum. In gradient descent, we differentiate to find the slope at a specific point and find out if the slope is negative or positive—you’re descending into the minimum of the cost function. There are several types of optimization strategies, but we’ll use a popular one known as adam in this tutorial.

In [11]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

Applying gradient descent is done via the compile function that takes the following parameters:

- optimizer is the gradient descent.
- loss is a function that we’ll use in the gradient descent. Since this is a binary classification problem we use the binary_crossentropy loss function.
- The last parameter is the metric that we’ll use to evaluate our model. In this case, we’d like to evaluate it based on its accuracy when making predictions.

In [12]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

1050/1050 [==============================] - 1s 927us/step - loss: 0.4382 - accuracy: 0.8188


We’re ready to fit our classifier to our dataset. Keras makes this possible via the .fit() method.

In [13]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

1050/1050 [==============================] - 1s 979us/step - loss: 0.2417 - accuracy: 0.9196


In [14]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

1050/1050 [==============================] - 1s 979us/step - loss: 0.1961 - accuracy: 0.9391


In [15]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)

1050/1050 [==============================] - 1s 866us/step - loss: 0.1779 - accuracy: 0.9471


The .fit() method takes a couple of parameters:

- The first parameter is the training set with the features.

- The second parameter is the column that we’re making the predictions on.

- The batch_size represents the number of samples that will go through the neural network at each training round.

- epochs represents the number of times that the dataset will be passed via the neural network. The more epochs the longer it will take to run our model, which also gives us better results.

We’ve created our deep learning model, compiled it, and fitted it to our dataset. You’re ready to make some predictions using the deep learning model.

Running Predictions on the Test Set

In [16]:
y_pred = classifier.predict(X_test)

Since we’ve already trained the classifier with the training set, this code will use the learning from the training process to make predictions on the test set. This will give us the probabilities of an employee leaving. We’ll work with a probability of 50% and above to indicate a high chance of the employee leaving the company.

In [17]:
## Setting threshold
y_pred = (y_pred > 0.5)

We’ve created predictions using the predict method and set the threshold for determining if an employee is likely to leave

Checking the Confusion Matrix

We will use a confusion matrix to check the number of correct and incorrect predictions. A confusion matrix, also known as an error matrix, is a square matrix that reports the number of true positives(tp), false positives(fp), true negatives(tn), and false negatives(fn) of a classifier.

- A true positive is an outcome where the model correctly predicts the positive class (also known as sensitivity or recall).
- A true negative is an outcome where the model correctly predicts the negative class.
- A false positive is an outcome where the model incorrectly predicts the positive class.
- A false negative is an outcome where the model incorrectly predicts the negative class.

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3321,   95],
       [ 142,  942]], dtype=int64)

The confusion matrix output means that our deep learning model made 3321 + 942 correct predictions and 95 + 142 wrong predictions. We can calculate the accuracy with: (3321 + 942) / 4500. The total number of observations in your dataset is 4500. This gives you an accuracy of 94.7%. This is a very good accuracy rate since we can achieve at least 94% correct predictions from our model.

Making a Single Prediction

We’ll make a single prediction given the details of one employee with our model. We will achieve this by predicting the probability of a single employee leaving the company. We’ll pass this employee’s features to the predict method. As we did earlier, we’ll scale the features as well and convert them to a numpy array.

In [19]:
new_pred = classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.]])))

These features represent the features of a single employee. As shown in the dataset in step 1, these features represent: satisfaction level, last evaluation, number of projects, and so on. As we did in step 3, we have to transform the features in a manner that the deep learning model can accept.

In [20]:
## Add a threshold of 50% with the following code:

new_pred = (new_pred > 0.5)
new_pred

array([[False]])

This threshold indicates that where the probability is above 50% an employee will leave the company.

You can see in our output that the employee won’t leave the company

In [21]:
## You might decide to set a lower or higher threshold for your model. For example, you can set the threshold to be 60%:

new_pred = (new_pred > 0.6)
new_pred

array([[False]])

This new threshold still shows that the employee won’t leave the company

Improving the Model Accuracy

If we train our model many times we’ll keep getting different results. The accuracies for each training have a high variance. In order to solve this problem, we’ll use K-fold cross-validation. Usually, K is set to 10. In this technique, the model is trained on the first 9 folds and tested on the last fold. This iteration continues until all folds have been used. Each of the iterations gives its own accuracy. The accuracy of the model becomes the average of all these accuracies.

keras enables us to implement K-fold cross-validation via the KerasClassifier wrapper. This wrapper is from scikit-learn cross-validation. We’ll start by importing the cross_val_score cross-validation function and the KerasClassifier.

In [22]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score


In [23]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

Here, we create a function that we’ll pass to the KerasClassifier—the function is one of the arguments that the classifier expects. The function is a wrapper of the neural network design that we used earlier. The passed parameters are also similar to the ones used earlier in the tutorial. In the function, we first initialize the classifier using Sequential(), we then use Dense to add the input and output layer. Finally, we compile the classifier and return it.

In [24]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

C:\Users\Convergytics\AppData\Local\Temp\ipykernel_2496\128404206.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)


The KerasClassifier takes three arguments:

- build_fn: the function with the neural network design
- batch_size: the number of samples to be passed via the network in each iteration
- nb_epoch: the number of epochs the network will run

In [25]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 10,n_jobs = -1)

This function will give us ten accuracies since we have specified the number of folds as 10. Therefore, we assign it to the accuracies variable and later use it to compute the mean accuracy. It takes the following arguments:

- estimator: the classifier that you’ve just defined
- X: the training set features
- y: the value to be predicted in the training set
- cv: the number of folds
- n_jobs: the number of CPUs to use (specifying it as -1 will make use of all the available CPUs)

In [26]:
## Now we have applied the cross-validation, we can compute the mean and variance of the accuracies. 

mean = accuracies.mean()
mean

0.823419040441513

In [27]:
## To compute the variance of the accuracies, add this code to the next notebook cell:

variance = accuracies.var()
variance

0.002355765038580735

Since the variance is very low, it means that our model is performing very well.

We’ve improved our model’s accuracy by using K-Fold cross-validation.

Adding Dropout Regularization to Fight Over-Fitting

Predictive models are prone to a problem known as overfitting. This is a scenario whereby the model memorizes the results in the training set and isn’t able to generalize on data that it hasn’t seen. Typically we observe overfitting when we have a very high variance on accuracies. To help fight over-fitting in our model, we will add a layer to our model.

In neural networks, dropout regularization is the technique that fights overfitting by adding a Dropout layer in our neural network. It has a rate parameter that indicates the number of neurons that will deactivate at each iteration. The process of deactivating nerurons is usually random. In this case, we specify 0.1 as the rate meaning that 1% of the neurons will deactivate during the training process. The network design remains the same.

In [28]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

We have added a Dropout layer between the input and output layer. Having set a dropout rate of 0.1 means that during the training process 15 of the neurons will deactivate so that the classifier doesn’t overfit on the training set. After adding the Dropout and output layers we then compiled the classifier as we have done previously.

We worked to fight over-fitting in this step with a Dropout layer.

Hyperparameter Tuning

Grid search is a technique that we can use to experiment with different model parameters in order to obtain the ones that give us the best accuracy. The technique does this by trying different parameters and returning those that give the best results. We’ll use grid search to search for the best parameters for our deep learning model. This will help in improving model accuracy. scikit-learn provides the GridSearchCV function to enable this functionality. We will now proceed to modify the make_classifier function to try out different parameters.

In [29]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

We have started by importing GridSearchCV. We have then made changes to the make_classifier function so that we can try different optimizers. We’ve initialized the classifier, added the input and output layer, and then compiled the classifier. Finally, we have returned the classifier so we can use it.

In [30]:
classifier = KerasClassifier(build_fn = make_classifier)

C:\Users\Convergytics\AppData\Local\Temp\ipykernel_2496\4215114608.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = make_classifier)


We’ve defined the classifier using the KerasClassifier, which expects a function through the build_fn parameter. We have called the KerasClassifier and passed the make_classifier function that we created earlier.

We will now proceed to set a couple of parameters that we wish to experiment with.

In [31]:
params = {
    'batch_size':[20,35],
    'epochs':[2,3],
    'optimizer':['adam','rmsprop']
}

Here we have added different batch sizes, number of epochs, and different types of optimizer functions.

For a small dataset like ours, a batch size of between 20–35 is good. For large datasets its important to experiment with larger batch sizes. Using low numbers for the number of epochs ensures that we get results within a short period. However, we can experiment with bigger numbers that will take a while to complete depending on the processing speed of our server. The adam and rmsprop optimizers from keras are a good choice for this type of neural network.

Now we’re going to use the different parameters we have defined to search for the best parameters using the GridSearchCV function

In [32]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

The grid search function expects the following parameters:

- estimator: the classifier that you’re using.
- param_grid: the set of parameters that you’re going to test.
- scoring: the metric you’re using.
- cv: the number of folds you’ll test on.

In [33]:
## Fit the grid search
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
263/263 [==============================] - 1s 938us/step - loss: 0.6012 - accuracy: 0.7506
Epoch 2/2
263/263 [==============================] - 0s 1ms/step - loss: 0.3955 - accuracy: 0.8457
Epoch 1/2
263/263 [==============================] - 1s 1ms/step - loss: 0.5989 - accuracy: 0.7636
Epoch 2/2
263/263 [==============================] - 0s 899us/step - loss: 0.4030 - accuracy: 0.8080
Epoch 1/2
263/263 [==============================] - 1s 1ms/step - loss: 0.5934 - accuracy: 0.7577
Epoch 2/2
263/263 [==============================] - 0s 853us/step - loss: 0.4245 - accuracy: 0.7859
Epoch 1/2
263/263 [==============================] - 1s 1ms/step - loss: 0.5946 - accuracy: 0.7672
Epoch 2/2
263/263 [==============================] - 0s 895us/step - loss: 0.4272 - accuracy: 0.8084
Epoch 1/3
263/263 [==============================] - 1s 1ms/step - loss: 0.5724 - accuracy: 0.7594
Epoch 2/3
263/263 [==============================] - 0s 1ms/step - loss: 0.3905 - accuracy: 0.8295
Ep

In [34]:
## Obtain the best parameters
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [35]:
## We can check the best parameters for our model
best_param

{'batch_size': 20, 'epochs': 3, 'optimizer': 'adam'}

Our output shows that the best batch size is 20, the best number of epochs is 2, and the adam optimizer is the best for our model.

In [36]:
## We can check the best accuracy for your model. The best_accuracy number represents the highest accuracy we obtain 
# from the best parameters after running the grid search.

best_accuracy

0.8616985003946329

We’ve used GridSearch to figure out the best parameters for our classifier. We have seen that the best batch_size is 20, the best optimizer is the adam optimizer and the best number of epochs is 3. We have also obtained the best accuracy for our classifier as being 86%. We’ve built an employee retention model that is able to predict if an employee stays or leaves with an accuracy of up to 86%.

##### Conclusion
We’ve used Keras to build an artificial neural network that predicts the probability that an employee will leave a company. To further improve our model, we can try different activation functions or optimizer functions from keras. We could also experiment with a different number of folds, or, even build a model with a different dataset.